Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
! pip install -q gliner pdfplumber python-docx PyMuPDF
from gliner import GLiNER

# Initialize GLiNER with the base model
model = GLiNER.from_pretrained("urchade/gliner_mediumv2.1")

import pdfplumber

with pdfplumber.open("test1.pdf") as pdf:
    page_texts = []

    for page in pdf.pages:
        text = page.extract_text()
        page_texts.append(text)

import docx
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

# More generic labels for legal documents
labels = [
    "Identifier",         
    "Location",          
    "Person",               
    "Organization",         
    "Date",  
    "Name", 
]


import pdfplumber
import fitz  # PyMuPDF

# Assuming model and other necessary imports are already in place
def split_into_chunks(text, chunk_size=50):
    words = text.split()
    for i in range(0, len(words), chunk_size):
        yield ' '.join(words[i:i+chunk_size])

def redact_pdf(input_pdf, output_pdf, redaction_dict):
    doc = fitz.open(input_pdf)
    
    for page in doc:
        for entity, label in redaction_dict.items():
            areas = page.search_for(entity)
            for rect in areas:
                page.add_redact_annot(rect, fill=(0, 0, 0))  # Black fill
        
        page.apply_redactions()
    
    doc.save(output_pdf)
    doc.close()

# Modify the main processing loop
redaction_dict = {}
with pdfplumber.open("test1.pdf") as pdf:
    for page_number, page in enumerate(pdf.pages, start=1):
        print(f"Processing Page {page_number}")
        page_text = page.extract_text()
        
        for chunk_number, chunk in enumerate(split_into_chunks(page_text), start=1):
            print(f"  Processing Chunk {chunk_number}")
            
            # Perform entity prediction
            entities = model.predict_entities(chunk, labels, threshold=0.5)
            
            # Add entities to redaction dictionary
            for entity in entities:
                entity_text = entity["text"]
                label = entity['label']
                redaction_dict[entity_text] = label

# Redact the PDF
input_pdf = "test1.pdf"
output_pdf = "redacted_test1.pdf"
redact_pdf(input_pdf, output_pdf, redaction_dict)

print(f"Redacted PDF has been saved as '{output_pdf}'")
print(f"\nSummary:")
print(f"Total entities redacted: {len(redaction_dict)}")
print("Redaction completed successfully.")
# Print summary


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Processing Page 1
  Processing Chunk 1
  Processing Chunk 2
  Processing Chunk 3
Processing Page 2
  Processing Chunk 1
  Processing Chunk 2
  Processing Chunk 3
Processing Page 3
  Processing Chunk 1
  Processing Chunk 2
  Processing Chunk 3
Processing Page 4
  Processing Chunk 1
  Processing Chunk 2
  Processing Chunk 3
Processing Page 5
  Processing Chunk 1
  Processing Chunk 2
  Processing Chunk 3
Processing Page 6
  Processing Chunk 1
  Processing Chunk 2
  Processing Chunk 3
Processing Page 7
  Processing Chunk 1
  Processing Chunk 2
  Processing Chunk 3
Processing Page 8
  Processing Chunk 1
  Processing Chunk 2
  Processing Chunk 3
Processing Page 9
  Processing Chunk 1
  Processing Chunk 2
  Processing Chunk 3
Processing Page 10
  Processing Chunk 1
  Processing Chunk 2
  Processing Chunk 3
Processing Page 11
  Processing Chunk 1
  Processing Chunk 2
  Processing Chunk 3
Processing Page 12
  Processing Chunk 1
  Processing Chunk 2
  Processing Chunk 3
Processing Page 13
  Proc